In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Lets import the libraries**

In [ ]:
import pandas as pd
import numpy as np
import math
import scipy
from sklearn.linear_model import LinearRegression


In [ ]:
price=pd.read_csv('/kaggle/input/real-estate-price-prediction/Real estate.csv')
price.head()

# ** clearly No column will not be at all useful, as it is just showing the index number, 
# so drop the No column**

In [ ]:
price.drop('No',axis=1,inplace=True)
price.head()

# **Column names are not written in proper way.
# There are spaces in between the column name.
# So lets fill the spaces by underscore(_) for smooth operation*********

In [ ]:
col_names=price.columns
col_names=col_names.str.replace(" ","_")
price.columns=col_names


price.columns
## column names got changed to the names we wanted

#  lets see are there any null values in the dataset ?

In [ ]:
price.isnull().sum()

## no null values in the data
## thats a good sign
## will make the code writing somewhat easier

In [ ]:
price.describe()

## in column starting with names X3 and X4
## the mean value of the column and its maximum value are having a huge difference between them
## so it means some outliers are there 
## lets deal with those outliers are the last
## hopefully it will boost our algorithms accuracy

**Lets see the data type of each column
if any data-type is of object(categorical) type, we need to change it to numerical type.**

In [ ]:
price.info()

## all columns have data type of int or float type

#  Data Visualization

In [ ]:
## lets visualize all the numeric columns

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,10))
sns.pairplot(price)
plt.show()

# **> Lets see the co-relation between the columns and the target column -> Price of house**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr=price.corr()
plt.figure(figsize=(10,6))
sns.heatmap(corr,annot=True)


# Ther's so much relation between the columns starting their name with X4, X5, X6 and also with the target column -> price of house column.


# **So lets use the traditional way to see which column will contribute towards the linear regression model and which are not contributing, we will discard that column**

In [ ]:
## Lets say the boundary line for alpha will be 0.05

## if p-value > alpha :  so that coumn is not contributing to the model


from statsmodels.formula.api import ols

model=ols(formula = 'Y_house_price_of_unit_area ~X1_transaction_date',data=price).fit()
model.summary()


***Since P>|t| for the first column X1 is greater than alpha=0.05.
So lets discard that column.
Since it will not be contributing to the model***

**Lets go on adding the other column to see their contribution in the model**

**Lets try some feature engineering ideas here.
Lets multiply longitude and latitude column

****
you can try addition or any other operation which will boost the accuracy.

**
As of now, lets go with multiplication idea.****

**so lets do the following to the dataset :

X1_transaction_date -> column

**replace latitude column** as -> X5_latitude * X6_longitude

X5_latitude and X6_longitude -> column*******

In [ ]:
price.drop('X1_transaction_date',axis=1,inplace=True)

price['X5_latitude']=price['X5_latitude'] * price['X6_longitude']

price.drop(['X6_longitude'],axis=1,inplace=True)



## lets see now how the corelation heatmap looks like


corr=price.corr()
plt.figure(figsize=(10,6))
sns.heatmap(corr,annot=True)


#  Data splitting into training and testing model

In [ ]:
from sklearn.model_selection import train_test_split

x = price.iloc[:,:-1]
y = price.iloc[:,-1]

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)



# Data modelling and evaluation

In [ ]:
linear_reg=LinearRegression()
model=linear_reg.fit(xtrain,ytrain)
y_predict=model.predict(xtest)


In [ ]:
## lets check the mean square error 

from sklearn.metrics import mean_squared_error

mean_squared_error(ytest,y_predict)

## low value of mean squared error
## thats a good sign

# since accuracy score doesnt go hand in hand with linear regression
#  so we will use r2_score function

In [ ]:
from sklearn.metrics import r2_score

r2_score(ytest,y_predict,)

## 67 % accuracy 

# ## lets see the actual plot between ytest and y_predicted values

In [ ]:
df = pd.DataFrame({'Actual':ytest,'Predictions':y_predict})
df['Predictions']= round(df['Predictions'])
df.tail()

In [ ]:
sns.regplot('Actual','Predictions',data=df)

## relatively good graph
## less deviation from main data line